# Tangent linear models and adjoint models: introduction

This notebook describes the TL and adjoint models for various very simple models, and implements the tangent model test and the adjoint model test. Both are recalled right below.

Most models used in this notebook are examples drawn from the lecture notes of my colleagues Arthur Vidard and Maëlle Nodet.

### Tangent model test

The tangent test checks the Taylor expansion at first order:

$$ f(x+\gamma dx) = f(x) + \gamma ~\nabla f (x).dx $$

where $\nabla f (x)$ is the Jacobian of $f$ taken at $x$. We compute the variable-wise ratios:
    
$$ \frac{f(x+\gamma dx) - f(x)}{\gamma ~\nabla f (x).dx} $$
    
with denominator computed with the tangent linear model as $f_{tl}(x, \gamma ~ dx)$. As $\gamma$ tends to 0, the ratios must tend to 1.

### Adjoint test

The adjoint test checks the following equality:

$$ < \nabla f (x).dx, dy > = < dx, \nabla f (x)^{*}.dy > $$

In [1]:
import numpy as np

# Model 1

The model simply is:

$$ Z = X \sin (Y^2) + a X^2 + bY +c $$

where $X, Y, Z$ are the active variables and $a, b, c$ are fixed parameters.

To make the exposition a little bit more general, we assume this function is step number $l$ in a process, transforming variables $X_{l-1}, Y_{l-1}, Z_{l-1}$ from step $l-1$ into $X_{l}, Y_{l}, Z_{l}$ at step $l$.

The tangent linear model between writes:

\begin{eqnarray}
\delta X_{l} &=&\delta X_{l-1} \\
\delta Y_{l} &=&\delta Y_{l-1} \\
\delta Z_{l} &=& \sin({Y_{l-1}}^2)~\delta X_{l-1}~ + ~2 ~X_{l-1}~Y_{l-1}~\cos({Y_{l-1}}^2)~\delta Y_{l-1}~+~2~a~X_{l-1}~\delta X_{l-1} ~+~ b~\delta Y_{l-1}
\end{eqnarray}

and the adjoint model:
\begin{eqnarray}
ADX_{l-1} &=& ADX_l ~+ ~ \left(~ \sin({Y_{l-1}}^2) ~+ ~2~a~X_{l-1} ~\right) ~ ADZ_l   \\
ADY_{l-1} &=& ADY_l ~+ ~ \left(~ 2 ~X_{l-1}~Y_{l-1}~\cos({Y_{l-1}}^2) ~+ ~b ~ \right) ~ADZ_l \\
ADZ_{l-1} &=& 0  
\end{eqnarray}

In [2]:
class Model1():
    
    def __init__(self):
        self.nvar = 3
        self.var = np.random.randn(3)
        self.param = np.array([1, 2, 1.5])
        
    def run(self, x_in):
        x = np.copy(x_in)
        x[2] = x_in[0] * np.sin(x_in[1]**2)       \
            + self.param[0] * x_in[0]**2    \
            + self.param[1] * x_in[1] + self.param[2]
        return x
    
    def run_tl(self, x_in, dx_in):
        dx = np.copy(dx_in)
        dx[2] = dx_in[0] * np.sin(x_in[1]**2) + 2 * x_in[0] * x_in[1] * np.cos(x_in[1]**2) * dx_in[1]      \
            + self.param[0] * 2 * x_in[0]*dx_in[0]    \
            + self.param[1] * dx_in[1]
        return dx
    
    def run_adj(self, x_in, adj_in):
        x_adj = np.copy(x_in)
        x_adj[0] = ( np.sin(x_in[1]**2) + 2*self.param[0]*x_in[0] ) * adj_in[2] + adj_in[0]
        x_adj[1] = ( 2*x_in[0]*x_in[1]*np.cos(x_in[1]**2) + self.param[1] ) * adj_in[2] + adj_in[1]
        x_adj[2] = 0
        return x_adj

### Create instance

In [3]:
m = Model1()

### Perform tangent model test

In [4]:
s = np.array([2., 1., -1.])
ss = np.random.randn(3)
for i in range(8):
    gamma = 10**(-i)
    ds = gamma * ss
    ratio = ( m.run(s+ds) - m.run(s) ) / m.run_tl(s, ds)
    print((gamma, ratio))

(1, array([1.        , 1.        , 1.06629914]))
(0.1, array([1.        , 1.        , 1.04755802]))
(0.01, array([1.        , 1.        , 1.00530834]))
(0.001, array([1.        , 1.        , 1.00053647]))
(0.0001, array([1.       , 1.       , 1.0000537]))
(1e-05, array([1.        , 1.        , 1.00000537]))
(1e-06, array([1.        , 1.        , 1.00000054]))
(1e-07, array([1.        , 1.        , 1.00000006]))


### Perform adjoint model test

In [5]:
sx = np.array([2., 1., -1.])
dsx = np.random.randn(3)
dsy = np.random.randn(3)
zz1, zz2 = m.run_tl(sx, dsx), m.run_adj(sx, dsy)
zz1@dsy, dsx@zz2

(10.87501436864407, 10.875014368644072)

# Model 2

The model equation is:

$$ Z = ZX + Y $$

with all variables active. Compared with Model1, here $Z$ is used to compute $Z$ at the next step. This will highlight the need to be careful with the algorithm steps.

The tangent linear model is:

\begin{eqnarray}
\delta X_{l} &=&\delta X_{l-1} \\
\delta Y_{l} &=&\delta Y_{l-1} \\
\delta Z_{l} &=& Z_{l-1}~\delta X_{l-1}~ + ~\delta Y_{l-1}~+~X_{l-1}~\delta Z_{l-1}
\end{eqnarray}

and the adjoint model:

\begin{eqnarray}
ADX_{l-1} &=& ADX_l ~+ ~ Z_l ~ ADZ_l   \\
ADY_{l-1} &=& ADY_l ~+ ~ ADZ_l \\
ADZ_{l-1} &=&  X_l ~ ADZ_l 
\end{eqnarray}

If these equations are implemented recursively, overwriting current variables, the last line must come in last position. If not, the first two lines will use $ADZ_{l-1}$ instead of $ADZ_{l}$ and the adjoint model will be incorrect.

In [6]:
class Model2():
    
    def __init__(self):
        self.nvar = 3
        self.var = np.random.randn(3)
        
    def run(self, x_in):
        x = np.copy(x_in)
        x[2] = x_in[0] * x_in[2] + x_in[1]
        return x
    
    def run_tl(self, x_in, dx_in):
        dx = np.copy(dx_in)
        dx[2] = x_in[2] * dx_in[0] + dx_in[1] + x_in[0]*dx_in[2]
        return dx
    
    def run_adj(self, x_in, adj_in):
        x_adj = np.copy(x_in)
        x_adj[0] = adj_in[0] + x_in[2] * adj_in[2]
        x_adj[1] = adj_in[1] + adj_in[2]
        x_adj[2] = x_in[0] * adj_in[2]
        return x_adj

### Create instance

In [7]:
m = Model2()

### Perform tangent model test

In [8]:
s = np.array([2., 1., -1.])
ss = np.random.randn(3)
for i in range(8):
    gamma = 10**(-i)
    ds = gamma * ss
    ratio = ( m.run(s+ds) - m.run(s) ) / m.run_tl(s, ds)
    print((gamma, ratio))

(1, array([1.        , 1.        , 0.82653826]))
(0.1, array([1.        , 1.        , 0.98265383]))
(0.01, array([1.        , 1.        , 0.99826538]))
(0.001, array([1.        , 1.        , 0.99982654]))
(0.0001, array([1.        , 1.        , 0.99998265]))
(1e-05, array([1.        , 1.        , 0.99999827]))
(1e-06, array([1.        , 1.        , 0.99999983]))
(1e-07, array([1.        , 1.        , 0.99999999]))


### Perform adjoint model test

In [9]:
sx = np.array([2., 1., -1.])
dsx = np.random.randn(3)
dsy = np.random.randn(3)
zz1, zz2 = m.run_tl(sx, dsx), m.run_adj(sx, dsy)
zz1@dsy, dsx@zz2

(-12.465707683833914, -12.465707683833916)

## Model 3

This model presents a loop:
$$ Y = F \times \prod_{i=1}^n X_i $$

The adjoint function requires the online recomputation of a parameter. Alternatives are to store values during a forward computation (in a file or an array).

In [10]:
class Model3():
        
    def run(self, F, X):
        """F, X: input. Y: output"""
        nx = len(X)
        Y = F
        for i in range(nx):
            Y *= X[i]
        return Y
    
    def run_tl(self, F, X, dFin, dXin):
        nx = len(X)
        Y, dY = F, dFin
        for i in range(nx):
            dY = X[i]*dY + Y*dXin[i]
            Y = Y*X[i]
        return dY
    
    def run_adj(self, F, X, adFin, adXin, adYin):
        """Adjoint model."""
        nx = len(X)
        adF, adX = adYin, adXin
        for i in range(nx-1,-1,-1):
            ## Compute Fout at appropriate iteration
            Fout = F
            for j in range(i):
                Fout = Fout*X[j]
            adX[i] += Fout*adF  ## need to compute Fout!
            adF = X[i]*adF
        adF += adFin
        return adF, adX

### Create instance

In [11]:
m = Model3()

In [12]:
s, F = np.array([2., 1., -1.]), 1.
m.run(F,s)

-2.0

### Perform tangent model test

In [13]:
s, F = np.array([2., 1., -1.]), 1.
ss, FF = np.random.randn(3), np.random.randn(1)
for i in range(8):
    gamma = 10**(-i)
    ds, dF = gamma * ss, gamma * FF
    ratio = ( m.run(F+dF, s+ds) - m.run(F,s) ) / m.run_tl(F, s, dF, ds)
    print((gamma, ratio))

(1, array([1.09539823]))
(0.1, array([1.02041166]))
(0.01, array([1.00215928]))
(0.001, array([1.00021712]))
(0.0001, array([1.00002172]))
(1e-05, array([1.00000217]))
(1e-06, array([1.00000022]))
(1e-07, array([1.00000002]))


### Perform adjoint model test

In [14]:
s, F = np.array([2., 1., -1.]), 3.
ds, dF = 0.1*np.random.randn(3), 0.1*np.random.randn(1)
dY = 0.1*np.random.randn(1)

zz1 = m.run_tl(F, s, dF, ds)
zz2 = m.run_adj(F, s, 0, np.zeros(3), dY)

zz1*dY , dF*zz2[0]+ds@zz2[1]

(array([0.01773841]), array([0.01773841]))